In [6]:
import pandas as pd 
df = pd.read_csv("GamePass_Games.csv", header=1)
print(df.head(5))

                          Game     System xCloud  Status     Added Removed  \
0        Bratz: Rhythm & Style  Xbox / PC    Yes  Active  Dec 2025     NaN   
1              Mortal Kombat 1  Xbox / PC    Yes  Active  Dec 2025     NaN   
2  A Game About Digging A Hole       Xbox    Yes  Active  Dec 2025     NaN   
3                   Death Howl         PC    NaN  Active  Dec 2025     NaN   
4                  Dome Keeper       Xbox    Yes  Active  Dec 2025     NaN   

   Months   Release   Age Metacritic  Completion             Genre Series X|S  \
0    0.27  Sep 2025  0.25        NaN         NaN  Action-Adventure        NaN   
1    0.30  Sep 2023  2.23         83         NaN  Action-Adventure  Exclusive   
2    0.33  Feb 2025  0.84        NaN         2.0         Adventure  Exclusive   
3    0.33  Dec 2025  0.00        NaN         NaN         Adventure        NaN   
4    0.33  Sep 2022  3.20         90         5.0            Action        NaN   

          Owner Notes ESRB ESRB Content Desc

In [5]:
df = df[df["Status"].isin(["Active", "Leaving Soon"])]
df.size


14360

In [ ]:
import pandas as pd
import re
import asyncio
import nest_asyncio
import os
from patchright.async_api import async_playwright

nest_asyncio.apply()

async def get_game_data_debug(game_name):
    async with async_playwright() as p:
        # 1. Launch Patchright in stealth mode
        browser = await p.chromium.launch(headless=False) 
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        )
        page = await context.new_page()

        for index, row in df_subset.iterrows():
            game_name = row['Game']
            print(f"[{index+1}/10] Investigating: {game_name}")
        # 2. Refined search query for better accuracy
        search_query = clean_game_name(game_name)
        
        # 2. FORCE THE XBOX DOMAIN
        # This tells DuckDuckGo: "Only show me results from the Xbox Store"
        search_url = f"https://duckduckgo.com/?q=site:xbox.com/en-us/games/store+{search_query}"
        
        try:
            print(f"Investigating: {game_name}")
            await page.goto(search_url)
            
            # 3. Wait for the page to settle
            await page.wait_for_timeout(5000) 


            # 5. Use Locator to find the first link that looks like a store result
            # We look for ANY link containing 'xbox.com' to be safe
            result_locator = page.locator('a[href*="xbox.com/en-us/games/store"]').first
            
            store_link = "Not Found"
            product_id = "N/A"

            # Check if the locator actually found something
            if await result_locator.count() > 0:
                store_link = await result_locator.get_attribute("href")
                match = re.search(r"([a-zA-Z0-9]{12})", store_link)
                product_id = match.group(1) if match else "ID Not Found"
            
            await browser.close()
            return store_link, product_id, screenshot_path
            
        except Exception as e:
            print(f"Error: {e}")
            await browser.close()
            return "Error", "N/A", "None"
        
        

# Run a test
# 2. Grab the first 10 rows
df_first_10 = df.head(10)

# 3. Run the scraper
final_results = await scrape_batch(df_first_10)

# 4. Save to a new CSV
results_df = pd.DataFrame(final_results)
results_df.to_csv('xbox_batch_results.csv', index=False)

print("\n--- Batch Complete! ---")
print(results_df)

[1/10] Investigating: Bratz: Rhythm & Style
[2/10] Investigating: Mortal Kombat 1
[3/10] Investigating: A Game About Digging A Hole
[4/10] Investigating: Death Howl
[5/10] Investigating: Dome Keeper
[6/10] Investigating: Routine
[7/10] Investigating: Lost Records: Bloom & Rage
[8/10] Investigating: Marvel Cosmic Invasion
[9/10] Investigating: Banishers: Ghosts of New Eden
[10/10] Investigating: Kill It With Fire! 2

--- Batch Complete! ---
                            Game  \
0          Bratz: Rhythm & Style   
1                Mortal Kombat 1   
2    A Game About Digging A Hole   
3                     Death Howl   
4                    Dome Keeper   
5                        Routine   
6     Lost Records: Bloom & Rage   
7         Marvel Cosmic Invasion   
8  Banishers: Ghosts of New Eden   
9           Kill It With Fire! 2   

                                       MS_Store_Link     ProductID  
0                                          Not Found           N/A  
1                    

In [ ]:
import pandas as pd
import re
import asyncio
import nest_asyncio
import os
import random
from patchright.async_api import async_playwright

nest_asyncio.apply()

async def scrape_batch(df_subset):
    results = []
    
    async with async_playwright() as p:
        # Launch stealth browser
        browser = await p.chromium.launch(headless=False) 
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        )
        page = await context.new_page()

        for index, row in df_subset.iterrows():
            game_name = row['Game']
            print(f"[{index+1}/10] Investigating: {game_name}")
            
            search_url = f"https://duckduckgo.com/?q={game_name}+xbox+store+links"
            
            try:
                await page.goto(search_url)
                # Wait for results to load
                await page.wait_for_timeout(random.randint(7000)) 

                # Grab the first Xbox store link
                result_locator = page.locator('a[href*="xbox.com/en-us/games/store"]').first
                
                store_link = "Not Found"
                product_id = "N/A"

                if await result_locator.count() > 0:
                    store_link = await result_locator.get_attribute("href")
                    # Remove any UTM parameters before regex
                    clean_link = store_link.split('?')[0]
                    match = re.search(r"([a-zA-Z0-9]{12})", clean_link)
                    product_id = match.group(1) if match else "ID Not Found"
                
                results.append({
                    "Game": game_name,
                    "MS_Store_Link": store_link,
                    "ProductID": product_id
                })
                
                # Human-like pause between games
                await asyncio.sleep(random.uniform(2, 4))
                
            except Exception as e:
                print(f"Error for {game_name}: {e}")
                results.append({"Game": game_name, "MS_Store_Link": "Error", "ProductID": "N/A"})

        await browser.close()
    return results





[1/10] Investigating: Bratz: Rhythm & Style
[2/10] Investigating: Mortal Kombat 1
Error for Mortal Kombat 1: Page.wait_for_timeout: Target page, context or browser has been closed
[3/10] Investigating: A Game About Digging A Hole
Error for A Game About Digging A Hole: Page.goto: Target page, context or browser has been closed
[4/10] Investigating: Death Howl
Error for Death Howl: Page.goto: Target page, context or browser has been closed
[5/10] Investigating: Dome Keeper
Error for Dome Keeper: Page.goto: Target page, context or browser has been closed
[6/10] Investigating: Routine
Error for Routine: Page.goto: Target page, context or browser has been closed
[7/10] Investigating: Lost Records: Bloom & Rage
Error for Lost Records: Bloom & Rage: Page.goto: Target page, context or browser has been closed
[8/10] Investigating: Marvel Cosmic Invasion
Error for Marvel Cosmic Invasion: Page.goto: Target page, context or browser has been closed
[9/10] Investigating: Banishers: Ghosts of New Ede

In [8]:
import pandas as pd
import re
import asyncio
import nest_asyncio
import random
import os
from patchright.async_api import async_playwright

nest_asyncio.apply()

# Helper function to clean names (Fixes the Bratz/Special Char issue)
def clean_game_name(name):
    clean = name.replace("&", "and")
    clean = re.sub(r'[^a-zA-Z0-9\s]', ' ', clean)
    return " ".join(clean.split())

async def scrape_xbox_batch(df_subset):
    results = []
    
    # 1. Start Patchright Stealth Browser
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False) 
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        )
        page = await context.new_page()

        # 2. Loop through your 10 games
        for index, row in df_subset.iterrows():
            game_name = row['Game']
            clean_name = clean_game_name(game_name)
            
            # 3. Targeted Search Query
            search_url = f"https://duckduckgo.com/?q={clean_name}+xbox+store+links"
            
            print(f"[{index+1}/10] Searching: {game_name}")
            
            try:
                await page.goto(search_url)
                # Wait for the page to render
                await page.wait_for_timeout(random.randint(4000, 6000)) 

                # 4. Grab the first organic store link
                result_locator = page.locator('a[href*="xbox.com/en-us/games/store"]').first
                
                store_link = "Not Found"
                product_id = "N/A"

                if await result_locator.count() > 0:
                    store_link = await result_locator.get_attribute("href")
                    # Snip the 12-char ID from the clean link
                    clean_link = store_link.split('?')[0]
                    match = re.search(r"([a-zA-Z0-9]{12})", clean_link.strip('/'))
                    product_id = match.group(1) if match else "ID Not Found"
                
                results.append({
                    "Game": game_name,
                    "MS_Store_Link": store_link,
                    "ProductID": product_id
                })
                
                # Human-like pause to prevent IP flagging
                await asyncio.sleep(random.uniform(2, 4))
                
            except Exception as e:
                print(f"Error on {game_name}: {e}")
                results.append({"Game": game_name, "MS_Store_Link": "Error", "ProductID": "N/A"})

        await browser.close()
    return results

# --- Main Execution ---
# Make sure your file name matches exactly!
df_first_10 = df.head(10)

# Run the scraper
batch_results = await scrape_xbox_batch(df_first_10)

# Save and View
results_df = pd.DataFrame(batch_results)
results_df.to_csv('xbox_batch_results.csv', index=False)
print("\n--- Batch Complete! ---")
print(results_df)

[1/10] Searching: Bratz: Rhythm & Style
[2/10] Searching: Mortal Kombat 1
[3/10] Searching: A Game About Digging A Hole
[4/10] Searching: Death Howl
[5/10] Searching: Dome Keeper
[6/10] Searching: Routine
[7/10] Searching: Lost Records: Bloom & Rage
[8/10] Searching: Marvel Cosmic Invasion
[9/10] Searching: Banishers: Ghosts of New Eden
[10/10] Searching: Kill It With Fire! 2

--- Batch Complete! ---
                            Game  \
0          Bratz: Rhythm & Style   
1                Mortal Kombat 1   
2    A Game About Digging A Hole   
3                     Death Howl   
4                    Dome Keeper   
5                        Routine   
6     Lost Records: Bloom & Rage   
7         Marvel Cosmic Invasion   
8  Banishers: Ghosts of New Eden   
9           Kill It With Fire! 2   

                                       MS_Store_Link     ProductID  
0  https://www.xbox.com/en-us/games/store/bratz-r...  9p76mmjg50ds  
1                                          Not Found         

In [10]:
import pandas as pd
import re
import asyncio
import nest_asyncio
import random
import os
from patchright.async_api import async_playwright

nest_asyncio.apply()

def clean_game_name(name):
    """Refines the game title for better search engine matching."""
    clean = name.replace("&", "and")
    clean = re.sub(r'[^a-zA-Z0-9\s]', ' ', clean)
    return " ".join(clean.split())

async def scrape_xbox_batch(df_subset):
    results_file = 'xbox_batch_results.csv'
    results = []
    
    async with async_playwright() as p:
        # Patchright stealth browser initialization
        browser = await p.chromium.launch(headless=False) 
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        )
        page = await context.new_page()

        for index, row in df_subset.iterrows():
            game_name = row['Game']
            clean_name = clean_game_name(game_name)
            
            # Using the 'site:' operator to force official store results
            search_url = f"https://duckduckgo.com/?q={clean_name}+xbox+store+links"
            
            print(f"[{index+1}/{len(df_subset)}] Searching: {game_name}")
            
            try:
                await page.goto(search_url)
                # Random wait to mimic human browsing behavior
                await page.wait_for_timeout(random.randint(5000, 7000)) 

                # Extracting all links to find the best match
                all_links = await page.eval_on_selector_all("a", "elements => elements.map(e => e.href)")
                
                store_link = "Not Found"
                product_id = "N/A"

                for link in all_links:
                    if "xbox.com/en-us/games/store/" in link:
                        # Clean tracking parameters before ID extraction
                        clean_link = link.split('?')[0].rstrip('/')
                        match = re.search(r"([a-zA-Z0-9]{12})$", clean_link)
                        
                        if match:
                            store_link = link
                            product_id = match.group(1)
                            break 

                current_result = {
                    "Game": game_name,
                    "MS_Store_Link": store_link,
                    "ProductID": product_id
                }
                results.append(current_result)
                
                # Save-as-you-go: Append this row to the CSV immediately
                pd.DataFrame([current_result]).to_csv(
                    results_file, 
                    mode='a', 
                    index=False, 
                    header=not os.path.exists(results_file)
                )
                
                await asyncio.sleep(random.uniform(2, 4))
                
            except Exception as e:
                print(f"Error on {game_name}: {e}")
                error_row = {"Game": game_name, "MS_Store_Link": "Error", "ProductID": "N/A"}
                results.append(error_row)
                pd.DataFrame([error_row]).to_csv(results_file, mode='a', index=False, header=False)

        await browser.close()
    return results

# --- Main Execution ---
# Ensure your CSV is in the same directory or provide the full path
df_first_10 = df.head(10)
await scrape_xbox_batch(df_first_10)
print("\n--- Batch Complete! Results saved to xbox_batch_results.csv ---")

[1/10] Searching: Bratz: Rhythm & Style
[2/10] Searching: Mortal Kombat 1
[3/10] Searching: A Game About Digging A Hole
[4/10] Searching: Death Howl
[5/10] Searching: Dome Keeper
[6/10] Searching: Routine
[7/10] Searching: Lost Records: Bloom & Rage
[8/10] Searching: Marvel Cosmic Invasion
[9/10] Searching: Banishers: Ghosts of New Eden
[10/10] Searching: Kill It With Fire! 2

--- Batch Complete! Results saved to xbox_batch_results.csv ---


In [16]:
import pandas as pd
import re
import asyncio
import nest_asyncio
import random
import os
from patchright.async_api import async_playwright

nest_asyncio.apply()

# 1. NEW: Logic to reset the file before the loop starts
results_file = 'xbox_batch_results.csv'
if os.path.exists(results_file):
    os.remove(results_file) # Deletes the old file so you start fresh

def clean_game_name(name):
    clean = name.replace("&", "and")
    clean = re.sub(r'[^a-zA-Z0-9\s]', ' ', clean)
    return " ".join(clean.split())

async def scrape_xbox_batch(df_subset):
    async with async_playwright() as p:
        # Patchright stealth browser
        browser = await p.chromium.launch(headless=False) 
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        )
        page = await context.new_page()

        for index, row in df.iterrows():
            game_name = row['Game']
            clean_name = clean_game_name(game_name)
            
            # 2. IMPROVED SEARCH: Force results from the official store
            search_url = f"https://duckduckgo.com/?q={clean_name}+xbox+store+links"
            
            print(f"[{index+1}/{len(df_subset)}] Searching: {game_name}")
            
            try:
                await page.goto(search_url)
                await page.wait_for_timeout(random.randint(5000, 7000)) 

                # Grab all links to ensure we find the right one
                all_links = await page.eval_on_selector_all("a", "elements => elements.map(e => e.href)")
                
                store_link = "Not Found"
                product_id = "N/A"

                for link in all_links:
                    if "xbox.com/en-us/games/store/" in link:
                        clean_link = link.split('?')[0].rstrip('/')
                        match = re.search(r"([a-zA-r0-9]{12})$", clean_link)
                        if match:
                            store_link = link
                            product_id = match.group(1)
                            break 

                current_result = {"Game": game_name, "MS_Store_Link": store_link, "ProductID": product_id}
                
                # Append to file
                pd.DataFrame([current_result]).to_csv(
                    results_file, 
                    mode='a', 
                    index=False, 
                    header=not os.path.exists(results_file)
                )
                
                await asyncio.sleep(random.uniform(2, 4))
                
            except Exception as e:
                print(f"Error on {game_name}: {e}")
                error_row = {"Game": game_name, "MS_Store_Link": "Error", "ProductID": "N/A"}
                pd.DataFrame([error_row]).to_csv(results_file, mode='a', index=False, header=False)

        await browser.close()

# --- Main Execution ---
# 2. Automatically find where you left off
start_index = 0
if os.path.exists(results_file):
    # Count rows in your existing results to find the starting point
    existing_df = pd.read_csv(results_file)
    start_index = len(existing_df)
    print(f"Resuming from row {start_index}...")

# 3. Create a subset of ONLY the remaining games
df_remaining = df.iloc[start_index:]

# 4. Run the scraper on ONLY the remaining rows
await scrape_xbox_batch(df_remaining)

print(f"\nBatch Complete! All results are in {results_file}")


[1/2158] Searching: Bratz: Rhythm & Style
Error on Bratz: Rhythm & Style: Page.wait_for_timeout: Target page, context or browser has been closed
[2/2158] Searching: Mortal Kombat 1
Error on Mortal Kombat 1: Page.goto: Target page, context or browser has been closed
[3/2158] Searching: A Game About Digging A Hole
Error on A Game About Digging A Hole: Page.goto: Target page, context or browser has been closed
[4/2158] Searching: Death Howl
Error on Death Howl: Page.goto: Target page, context or browser has been closed
[5/2158] Searching: Dome Keeper
Error on Dome Keeper: Page.goto: Target page, context or browser has been closed
[6/2158] Searching: Routine
Error on Routine: Page.goto: Target page, context or browser has been closed
[7/2158] Searching: Lost Records: Bloom & Rage
Error on Lost Records: Bloom & Rage: Page.goto: Target page, context or browser has been closed
[8/2158] Searching: Marvel Cosmic Invasion
Error on Marvel Cosmic Invasion: Page.goto: Target page, context or browse

CancelledError: 